In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier


In [138]:
matches_df = pd.read_csv('../Datasets/final_feature_dataset.csv')
matches_df.head()
# matches_df.columns
# matches_df.shape

,Season,Home Team,Away Team,Home Goals,Away Goals,Home Team ELO,Away Team ELO,Home XG,Away XG,Home XGA,Away XGA,Home Win Percentage,Home Draw Percentage,Away Win Percentage,Away Draw Percentage,Home Team Form,Away Team Form,Home Team Cumulative Points,Away Team Cumulative Points,Winner
0,2014-2015,Manchester Utd,Swansea City,1,2,1676,1521,54.21,33.84,40.90,55.94,64.50,20.64,29.82,28.95,0.0,0.0,0,0,-1
1,2014-2015,West Brom,Sunderland,2,2,1485,1520,38.63,37.45,53.75,51.46,24.01,26.97,26.52,25.10,0.0,0.0,0,0,0
2,2014-2015,Leicester City,Everton,2,2,1462,1692,48.21,44.89,56.39,46.18,27.27,29.22,36.44,28.54,0.0,0.0,0,0,0
3,2014-2015,West Ham,Tottenham,0,1,1509,1680,46.69,57.40,52.39,57.04,32.51,25.29,39.62,25.83,0.0,0.0,0,0,-1
4,2014-2015,QPR,Hull City,0,1,1420,1434,45.73,65.26,32.79,46.99,30.42,24.58,21.05,26.32,0.0,0.0,0,0,-1


In [139]:
X = matches_df.drop(["Winner","Season",'Home Goals','Away Goals',"Home Team","Away Team"], axis=1)
y = matches_df["Winner"]

# categorical_columns = ["Home Team", "Away Team"]
# numerical_columns = [col for col in X.columns if col not in categorical_columns]

label_mapping = {-1: 0, 0: 1, 1: 2}
y_mapped = y.map(label_mapping)

X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.2, random_state=42)

In [140]:
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='sigmoid'),
    Dense(64, activation='sigmoid'),
    Dense(32, activation='sigmoid'),
    Dense(3, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1000, batch_size=64)

Epoch 1/1000


c:\Users\vimal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3104 - loss: 1.5354 - val_accuracy: 0.3197 - val_loss: 1.3948
Epoch 2/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3073 - loss: 1.3351 - val_accuracy: 0.3197 - val_loss: 1.2212
Epoch 3/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3202 - loss: 1.1848 - val_accuracy: 0.3197 - val_loss: 1.1232
Epoch 4/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3132 - loss: 1.1114 - val_accuracy: 0.3197 - val_loss: 1.0747
Epoch 5/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3953 - loss: 1.0759 - val_accuracy: 0.5553 - val_loss: 1.0525
Epoch 6/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5040 - loss: 1.0599 - val_accuracy: 0.4684 - val_loss: 1.0429
Epoch 7/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4487 - loss: 1.0576 - val_accuracy: 0.4684 - val_loss: 1.0384
Epoch 8/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4402 - loss: 1.0577 - val_accuracy: 0.4684 - val_lo

In [141]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step - accuracy: 0.7072 - loss: 0.6959
Test Loss: 0.7403786182403564
Test Accuracy: 0.6684210300445557


In [146]:
# model = Sequential([
#     Dense(128, input_dim=X_train.shape[1], activation='sigmoid'),
#     Dense(64, activation='sigmoid'),
#     Dense(32, activation='sigmoid'),
#     Dense(3, activation='softmax')
# ])

# model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1000, batch_size=64)
def create_model(optimizer='adam', activation='sigmoid'):
    model = Sequential([
        Dense(128, input_dim=X_train.shape[1], activation=activation),
        Dense(64, activation=activation),
        Dense(32, activation=activation),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, epochs=1000, batch_size=64, verbose=0)


In [147]:
param_grid = {
    'optimizer': ['Adam'],
    'epochs': [1000],
    'batch_size': [64]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

c:\Users\vimal\AppData\Local\Programs\Python\Python312\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\vimal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best: 0.5851858125758954 using {'batch_size': 64, 'epochs': 1000, 'optimizer': 'Adam'}
